# install stuff

In [1]:
! pip3 install pyspark pandas scikit-learn emoji #check-swear

# setup pyspark

In [2]:
import pyspark.sql
from pyspark.sql import functions as sf
import pyspark
import emoji
# from check_swear import SwearingCheck
import re
import pandas as pd



In [3]:
sparkConf = pyspark.SparkConf() \
    .set("spark.executor.memory", "10g")\
    .set("spark.driver.memory", "10g")\
    .set("spark.driver.maxResultSize", "8g")
ss = pyspark.sql.SparkSession.builder.config(conf=sparkConf).master('local[1]').getOrCreate()

24/06/09 02:39:46 WARN Utils: Your hostname, owner-All-Series resolves to a loopback address: 127.0.1.1; using 192.168.5.109 instead (on interface eno1)
24/06/09 02:39:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/09 02:39:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# load data

In [4]:
train_data = ss.read.orc('./hw_data/posts_train.orc')
test_data = ss.read.orc('./hw_data/posts_test.orc')
channel_data = ss.read.orc('./hw_data/channels_orc')

In [5]:
train_data.show()

+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+
|channel_id|      id|tg_id|                text|views|has_image|is_forwarded|                date|forwarded_id|
+----------+--------+-----+--------------------+-----+---------+------------+--------------------+------------+
|       221|29741094| 7182|МК, а это вот про...|15368|        f|           f|2018-11-03 13:05:...|        NULL|
|       221|46751120| 7388|Хочу пошутить, чт...|12448|        f|           f|2018-11-26 16:15:...|        NULL|
|       221|32631368| 5512|Уточню: Котляр бы...|69984|        f|           f|2018-05-16 16:03:...|        NULL|
|       221|46751758| 7173|Да не на два лаге...|10241|        f|           f|2018-11-02 13:45:...|        NULL|
|       221|33073441| 3185|А что касается Уд...|46847|        f|           f|2017-08-10 13:44:...|        NULL|
|       221|46751793| 7141|Кашин давно меня ...|    0|        f|           f|2018-10-30 22:04:...|      

# make features

In [52]:
from pyspark.sql.types import BooleanType, IntegerType
import pyspark.sql.functions as fn
# sch = SwearingCheck()

contains_emoji = fn.udf(lambda s: False if s is None else any(map(lambda x: x in s, emoji.EMOJI_DATA.keys())), 
                           BooleanType())
to_bool = fn.udf(lambda s: None if s is None else True if s == 't' else False)
no_symbols = fn.udf(lambda s: None if s is None else re.sub('[!?;,.]', '', s))
count_unique_words = fn.udf(lambda s: 0 if s is None else len(set(s.split(" "))) - 1)
count_words = fn.udf(lambda s: 0 if s is None else len(s.split(" ")) - 1)
# count_sentences = fn.udf(lambda s: 0 if s is None else len(s.split(".")) - 1 if )

@fn.udf(returnType=IntegerType())
def count_sentences(s):
    if s is None: return None
    l = len(s.split("."))
    if l == 0:
        if len(s) == 0: return 0
        else: return 1
    else: return l

def NORMAL_OHE(df, cols):
    for col_nm in cols:
        categories = df.select(col_nm).distinct().rdd.flatMap(lambda x : x).collect()
        categories.sort()
        for category in categories[:-1]:
            function = fn.udf(lambda item: 1 if item == category else 0, IntegerType())
            new_column_name = f'{col_nm}_{category}'
            df = df.withColumn(new_column_name, function(fn.col(col_nm)))
        df = df.drop(col_nm)
    return df




def make_features(df):
    
    df = df.withColumn('Literate', df.text.contains(", "))
    df = df.withColumn('Screaming', df.text.contains("!"))
    df = df.withColumn('Question', df.text.contains("?"))
    df = df.withColumn('Num_sentences', count_sentences(df.text))
    df = df.withColumn('Num_words', count_words(no_symbols(df.text)))
    df = df.withColumn("Distinct_words", count_unique_words(no_symbols(df.text)))
    # df= df.withColumn("Emoji_in_post", contains_emoji(fn.col("text")))
    df = df.withColumn("Length_of_text", fn.length(df.text))
    df = df.withColumn("has_image", to_bool(df.has_image))
    df = df.withColumn("is_forwarded", to_bool(df.is_forwarded))

    df = df.withColumn("year", fn.year(df.date))
    df = df.withColumn("month", fn.month(df.date))
    df = df.withColumn("day", fn.dayofmonth(df.date))
    df = df.withColumn("day_week", fn.dayofweek(df.date))
    df = df.withColumn("hour", fn.hour(df.date))
    df = df.withColumn("minute", fn.minute(df.date))
    df = df.withColumn("second", fn.second(df.date))
    df = df.drop("date")

    df = df.withColumn("forwarded_id", fn.isnotnull(df.forwarded_id))
    df = df.fillna(False, subset=['Literate', 'Screaming', 'Question'])
    df = df.fillna(0, subset=['Num_sentences', 'Length_of_text'])
    
    df =df.dropna()
    df = NORMAL_OHE(df, cols = ['year', 'month', 'day', 'day_week', 'hour', 'minute', 'second'])

    return df

In [53]:
train_features = make_features(train_data)#.cache()
test_features = make_features(test_data)#.cache()
train_features.show()

+----------+--------+-----+--------------------+-----+---------+------------+------------+--------+---------+--------+-------------+---------+--------------+--------------+---------+---------+---------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+----------+----------+----------+----------+----------+----------+------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+-------

In [ ]:
train_features.write.csv('train_csv', mode='overwrite', header=True)
test_features.write.csv('test_csv', mode='overwrite', header=True)

AnalysisException: [UNSUPPORTED_DATA_TYPE_FOR_DATASOURCE] The CSV datasource doesn't support the column `SPLIT` of the type "ARRAY<STRING>".

# load features to pandas
you also can use .toPandas()

In [ ]:
import subprocess
import glob
import os
import shutil
import pandas as pd

def load_and_merge_csv(path, **kwargs):
    dfs = []
    for g in glob.glob(os.path.join(path, '*.csv')):
        dfs.append(pd.read_csv(g, **kwargs))
    res = pd.concat(dfs)
    res = res.set_index('id')
    return res

In [ ]:
trainXY = train_features.toPandas()#load_and_merge_csv_from_hdfs('train_csv')
testX = test_features.toPandas()#load_and_merge_csv_from_hdfs('test_csv')

Py4JJavaError: An error occurred while calling o46.collectToPython.
: java.lang.OutOfMemoryError: Java heap space
	at scala.collection.mutable.ResizableArray.ensureSize(ResizableArray.scala:106)
	at scala.collection.mutable.ResizableArray.ensureSize$(ResizableArray.scala:96)
	at scala.collection.mutable.ArrayBuffer.ensureSize(ArrayBuffer.scala:49)
	at scala.collection.mutable.ArrayBuffer.$plus$eq(ArrayBuffer.scala:85)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect$2(SparkPlan.scala:449)
	at org.apache.spark.sql.execution.SparkPlan$$Lambda/0x000078f814dce578.apply(Unknown Source)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at org.apache.spark.util.NextIterator.foreach(NextIterator.scala:21)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect$1(SparkPlan.scala:449)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeCollect$1$adapted(SparkPlan.scala:448)
	at org.apach

In [ ]:
trainXY

,views
id,
46044656,14813
46044647,8499
46044648,10061
46044649,10672
46044650,11302
...,...
50140480,1664
50140481,1666
50140482,1680


In [ ]:
import numpy as np

Ycol = 'views'
to_drop = ['channel_id']
trainX, trainY = trainXY.drop(Ycol, axis=1).drop(to_drop, axis=1), trainXY[Ycol]
trainY = np.log(trainY + 100)

testX = testX.drop(to_drop, axis=1)

KeyError: "['channel_id'] not found in axis"

# train your model and predict test

In [ ]:
prediction = np.random.random(testX.shape[0])

In [ ]:
prediction.shape

(244386,)

In [ ]:
assert prediction.shape == (244386,)

# submit

In [ ]:
! curl -H 'Cache-Control: no-cache' https://raw.githubusercontent.com/mike0sv/lsml_submit_server/2024/src/client.py -o client.py

In [ ]:
import client

In [ ]:
client.make_eval(pd.DataFrame({'views': prediction}, index=testX.index))

In [ ]:
client.check_results()